In [1]:
import os
import boto3
from s3ssp import get_file,upload_file
import pandas as pd
import numpy as np
import re
import random
import spacy
import sklearn
import nltk

In [2]:
s3 = boto3.resource('s3')

In [3]:
lyrics = get_file(s3,'s3ssp',download_file='NLP_Data/rough_lyrics_audio_features.csv',rename_file='06_2020_nlp.csv')

In [4]:
df = pd.read_csv("06_2020_nlp.csv",sep='|',encoding='utf-8')

df = df.drop_duplicates(['artist_uri_x','track_uri'])
df_copy = df.copy().dropna()

#Running on this variable
df_sample = df_copy.head(10)

In [6]:
#Clean dataset of instrumentals
df_copy = df_copy.copy()

df_copy['instrumental'] = df_copy['lyrics'].apply(lambda x: x.find('instrumental'))
df_copy = df_copy.drop(df_copy[df_copy['instrumental'] != -1].index)
#df = df.drop(df[df['lyrics']=='This is an instrumental'].index)

In [7]:
df_copy['Instrumental'] = df_copy['lyrics'].apply(lambda x: x.find('Instrumental'))
df_copy =df_copy.drop(df_copy[df_copy['Instrumental'] != -1].index)

In [8]:
df_copy['INSTRUMENTAL'] = df_copy['lyrics'].apply(lambda x: x.find('INSTRUMENTAL'))
df_copy =df_copy.drop(df_copy[df_copy['INSTRUMENTAL'] != -1].index)

In [10]:
df_copy[df_copy['lyrics'] == 'INSTRUMENTAL']

,artist_uri_x,artist_name,track_uri,track_name,lyrics,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,instrumental,Instrumental,INSTRUMENTAL


In [11]:
lyrics_list = df_copy['lyrics'].str.replace('\n',' ').str.replace('\d+', '').str.lower()
df_copy['lyrics'] = lyrics_list.to_frame()

In [13]:
import spacy
regexp = re.compile('(?u)\\b\\w\\w+\\b')
en_nlp = spacy.load('en_core_web_sm',disable=['parser','ner'])
old_tokenizer = en_nlp.tokenizer
en_nlp.tokenizer = lambda string: old_tokenizer.tokens_from_list(regexp.findall(string))

In [14]:
def d_preprocessor(lyric):
        doc_spacy = en_nlp(lyric)
        lemma_lyric = [token.lemma_ for token in doc_spacy]
        lemma = ' '.join(lemma_lyric)
        return lemma.lower()

In [17]:
df_copy['lemma'] = df_copy['lyrics'].apply(lambda x: d_preprocessor(x))

In [59]:
#df_copy.head()

In [42]:
df_lastfm_stopwords = pd.read_csv('stopwords.txt',header=None)
lastfm_stopwords = df_lastfm_stopwords[0].str.replace('\n',' ')
df_lastfm_stopwords['last_fm_stopwords'] = lastfm_stopwords.to_frame()
df_lastfm_stopwords_list = df_lastfm_stopwords['last_fm_stopwords'].tolist()

df_english = pd.read_csv("english.txt",header=None)
github_english_stopwords = df_english[0].str.replace('\n',' ')
df_english['github_english_stopwords'] = github_english_stopwords.to_frame()
github_english_stopwords_list = df_english['github_english_stopwords'].tolist()

df_spanish = pd.read_csv("spanish.txt",header=None)
github_spanish_stopwords = df_spanish[0].str.replace('\n',' ')
df_spanish['github_spanish_stopwords'] = github_spanish_stopwords.to_frame()
github_spanish_stopwords_list = df_spanish['github_spanish_stopwords'].tolist()

df_spanish = pd.read_csv("spanish.txt",header=None)
github_spanish_stopwords = df_spanish[0].str.replace('\n',' ')
df_spanish['github_spanish_stopwords'] = github_spanish_stopwords.to_frame()
github_spanish_stopwords_list = df_spanish['github_spanish_stopwords'].tolist()

df_german = pd.read_csv("german.txt",header=None)
github_german_stopwords = df_german[0].str.replace('\n',' ')
df_german['github_german_stopwords'] = github_german_stopwords.to_frame()
github_german_stopwords_list = df_german['github_german_stopwords'].tolist()

df_french = pd.read_csv("french.txt",header=None)
github_french_stopwords = df_french[0].str.replace('\n',' ')
df_french['github_french_stopwords'] = github_french_stopwords.to_frame()
github_french_stopwords_list = df_french['github_french_stopwords'].tolist()

df_two_word = pd.read_csv("two_word_stopwords.csv",header=None)
two_word_stopwords = df_two_word[0].str.replace('\n',' ')
df_two_word['two_word_stopwords'] = two_word_stopwords.to_frame()
two_word_stopwords_list = df_two_word['two_word_stopwords'].tolist()

In [43]:
df_lastfm_stopwords.head()

,0,last_fm_stopwords
0,a,a
1,able,able
2,about,about
3,above,above
4,abst,abst


In [44]:
github_french_stopwords_list

['a',
 'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'alors',
 'as',
 'au',
 'aucun',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aussi',
 'autre',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avant',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avoir',
 'avons',
 'ayant',
 'ayez',
 'ayons',
 'bon',
 'car',
 'ce',
 'ceci',
 'cela',
 'ces',
 'cet',
 'cette',
 'ceux',
 'chaque',
 'ci',
 'comme',
 'comment',
 'd',
 'dans',
 'de',
 'dedans',
 'dehors',
 'depuis',
 'des',
 'deux',
 'devoir',
 'devrait',
 'devrez',
 'devriez',
 'devrions',
 'devrons',
 'devront',
 'dois',
 'doit',
 'donc',
 'dos',
 'droite',
 'du',
 'dès',
 'début',
 'dù',
 'elle',
 'elles',
 'en',
 'encore',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'faire',
 'fais',
 'faisez',
 'fait',
 'faites',
 'fois',
 'font',
 'force',
 'furent',

In [45]:
df_demo = df_copy
df_test=df_copy.head(10)

In [46]:
from nltk import word_tokenize
from nltk.corpus import stopwords

def preprocessor(text,source):
    tokens = word_tokenize(text)
    return (" ").join([word for word in tokens if word not in source and word != '-pron-' and word.isalpha()])
            
    #return (" ").join([word for word in if word.lower() not in source])

In [47]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/navneetsandhu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/navneetsandhu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
df_demo['git_english_lyrics'] = df_demo['lemma'].apply(lambda x: preprocessor(x,github_english_stopwords_list))
df_demo['git_spanish_lyrics'] = df_demo['git_english_lyrics'].apply(lambda x: preprocessor(x,github_spanish_stopwords_list))

In [49]:
df_demo['last_fm_lyrics'] = df_demo['git_spanish_lyrics'].apply(lambda x: preprocessor(x,df_lastfm_stopwords_list))
df_demo['nltk_english_lyrics'] = df_demo['last_fm_lyrics'].apply(lambda x: preprocessor(x,stopwords.words('english')))
df_demo['git_german_lyrics'] = df_demo['nltk_english_lyrics'].apply(lambda x: preprocessor(x,github_german_stopwords_list))
df_demo['git_french_lyrics'] = df_demo['git_german_lyrics'].apply(lambda x: preprocessor(x,github_french_stopwords_list))
df_demo['two_word_lyrics'] = df_demo['git_french_lyrics'].apply(lambda x: preprocessor(x,two_word_stopwords_list))
df_demo['nltk_spanish_lyrics'] = df_demo['two_word_lyrics'].apply(lambda x: preprocessor(x,stopwords.words('spanish')))

In [50]:
df_demo

,artist_uri_x,artist_name,track_uri,track_name,lyrics,acousticness,danceability,energy,instrumentalness,key,...,INSTRUMENTAL,lemma,git_english_lyrics,git_spanish_lyrics,last_fm_lyrics,nltk_english_lyrics,git_german_lyrics,git_french_lyrics,two_word_lyrics,nltk_spanish_lyrics
0,6vWDO969PvNqNYHIOW5v0m,Beyoncé,5Ui8M6tfknhXo4MuGHt3Dy,why don't you love me,"n-n-now, honey you better sit down and look ar...",0.046300,0.693,0.730,0.000002,6.0,...,-1,now honey -pron- better sit down and look arou...,honey sit bump yo head love talk sense baby ha...,honey sit bump head love talk sense baby hate ...,honey sit bump head love talk sense baby hate ...,honey sit bump head love talk sense baby hate ...,honey sit bump head love talk sense baby hate ...,honey sit bump head love talk sense baby hate ...,honey sit bump head love talk sense baby hate ...,honey sit bump head love talk sense baby hate ...
1,6vWDO969PvNqNYHIOW5v0m,Beyoncé,5dhPqcLr5EcSd7Fe4fslCq,save the hero,i lay alone awake at night sorrow fills my eye...,0.673000,0.551,0.467,0.000000,10.0,...,-1,lay alone awake at night sorrow fill -pron- ey...,lay awake night sorrow eye strong disguise lea...,lay awake night sorrow eye strong disguise lea...,lay awake night sorrow eye strong disguise lea...,lay awake night sorrow eye strong disguise lea...,lay awake night sorrow eye strong disguise lea...,lay awake night sorrow eye strong disguise lea...,lay awake night sorrow eye strong disguise lea...,lay awake night sorrow eye strong disguise lea...
2,6vWDO969PvNqNYHIOW5v0m,Beyoncé,5dWTQXVHdoIsSLpEyS3woy,broken-hearted girl,youre everything i thought you never were and ...,0.503000,0.336,0.424,0.000000,5.0,...,-1,youre everything think -pron- never be and not...,couldve live forget love forgive break heart t...,couldve live forget love forgive break heart t...,couldve live forget love forgive break heart t...,couldve live forget love forgive break heart t...,couldve live forget love forgive break heart t...,couldve live forget love forgive break heart t...,couldve live forget love forgive break heart t...,couldve live forget love forgive break heart t...
3,6vWDO969PvNqNYHIOW5v0m,Beyoncé,0QRxJvOohS8yiGC1n98uFM,lift ev'ry voice and sing - homecoming live,he is always laughin' and flirting with me and...,0.124000,0.127,0.409,0.000002,2.0,...,-1,-pron- be always laughin and flirt with -pron-...,laughin flirt care matter real distance actin ...,laughin flirt care matter real distance actin ...,laughin flirt care matter real distance actin ...,laughin flirt care matter real distance actin ...,laughin flirt care matter real distance actin ...,laughin flirt care matter real distance actin ...,laughin flirt care matter real distance actin ...,laughin flirt care matter real distance actin ...
4,6vWDO969PvNqNYHIOW5v0m,Beyoncé,4DActPOAtak2m8meZeMt3B,no angel,"this is for my fans (uhu, uhu) this is for my ...",0.042400,0.571,0.466,0.002620,9.0,...,-1,this be for -pron- fan uhu uhu this be for -pr...,fan uhu uhu destiny uhu uhu fan uhu uhu future...,fan uhu uhu destiny uhu uhu fan uhu uhu future...,fan uhu uhu destiny uhu uhu fan uhu uhu future...,fan uhu uhu destiny uhu uhu fan uhu uhu future...,fan uhu uhu destiny uhu uhu fan uhu uhu future...,fan uhu uhu destiny uhu uhu fan uhu uhu future...,fan uhu uhu destiny uhu uhu fan uhu uhu future...,fan uhu uhu destiny uhu uhu fan uhu uhu future...
6,6vWDO969PvNqNYHIOW5v0m,Beyoncé,63FrXif0Pdu4NAPvTh87mw,mine (feat. drake),[verse : beyonce] i've been watching for the s...,0.067300,0.557,0.428,0.001900,5.0,...,-1,verse beyonce ve be watch for the sign take tr...,verse beyonce watch sign trip mind lost fine c...,verse beyonce watch sign trip mind lost fine c...,verse beyonce watch sign trip mind lost fine c...,verse beyonce watch sign trip mind lost fine c...,verse beyonce watch sign trip mind lost fine c...,verse beyonce watch sign trip mind lost fine c...,verse beyonce watch sign trip mind lost fine c...,verse beyonce watch sign trip mind lost fine c...
7,6vWDO96

In [54]:
df_topic = df_demo[['artist_name','track_name','track_uri','acousticness','danceability','energy','valence'
                    ,'instrumentalness','key','mode','liveness','speechiness','time_signature',
                    'tempo','loudness','nltk_spanish_lyrics']]
df_topic = df_topic.rename(columns={'nltk_spanish_lyrics':'lyrics'})

In [56]:
df_topic['lyrics'].values

array(['honey sit bump head love talk sense baby hate kid dog check credential friend woman love baby love damn easy love baby damn easy beauty class style ass care care looka money bank account notice love baby love damn easy love baby damn easy beauty heart head book sharp care smart bedroom happy nasty tune ooh love baby love damn easy love baby damn easy love love lovely plain dumb',
       'lay awake night sorrow eye strong disguise leave shoulder lean guess soldier gon save hero leave save hero save girl save save bottle hurt guess living lie mind day bring life simple word gesture beautiful bury treasure eye lead pot gold save hero leave save hero save girl save save drive crazy save save save hero leave save hero save girl save save',
       'couldve live forget love forgive break heart time hate erase time hurt tear hate pain day wan babe break heart wan breath babe wan play love love break heart wan play break hearte girlno broken hearted girl broken hearted girl feel afraid 

In [57]:
df_topic.to_csv('master_lyrics_audio_features.csv',index=False,sep='|')

In [58]:
upload_file(s3,'s3ssp',upload_file='master_lyrics_audio_features.csv',rename_file='NLP_Data/master_lyrics_audio_features.csv')

'NLP_Data/master_lyrics_audio_features.csv'